In [2]:
import matplotlib.pyplot as plt
import numpy as np

from scipy.signal import convolve2d as conv2
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import Angle
import udft

from surfh.Simulation import simulation_data
from surfh.DottestModels import MCMO_SigRLSCT_Model
from surfh.ToolsDir import utils

from surfh.Simulation import fusion_CT
from surfh.Models import instru
from surfh.ToolsDir import fusion_mixing

from surfh.Models import wavelength_mrs, realmiri

import pathlib

In [3]:
"""
Create Model and simulation
"""
origin_alpha_axis, origin_beta_axis, wavel_axis, spsf, maps, templates = simulation_data.get_simulation_data(4, 0, '../cube_orion/') # subsampling to reduce dim of maps

wavelength_1c = wavelength_mrs.get_mrs_wavelength('1c')

2


In [4]:
with fits.open('/home/nmonnier/Data/JWST/Orion_bar/Stage_2/jw01288002001_0211f_00001_mirifushort_cal.fits') as hdul:
    hdr = hdul[1].header
    targ_ra  = hdr['RA_V1']
    targ_dec = hdr['DEC_V1']


orgin_target = instru.Coord(targ_ra, targ_dec)
print(targ_ra, targ_dec)

# ch1c = instru.IFU(
#     fov=instru.FOV(3.2/3600, 3.7/3600, origin=orgin_target, angle=8.2),
#     det_pix_size=0.196,
#     n_slit=21,
#     w_blur=None,
#     pce=None,
#     wavel_axis=wavelength_1c,
#     name="1C",
# )
ch1c, targ_ra, targ_dec = realmiri.get_IFU('/home/nmonnier/Data/JWST/Orion_bar/Stage_3/ChannelCube_ch1-long_s3d.fits')
step = 0.025
step_angle = Angle(step, u.arcsec).degree
ch1c_pix = ch1c.pix(step_angle)
print(targ_ra, targ_dec)

2024-08-09 10:57:20.058 | INFO     | surfh.Models.instru:pix:684 - IFU. 1C pixelized to 6.9e-06


83.98088399122913 -5.503855979171915
83.83536208333332 -5.41972777777778


In [10]:
# Super resolution factor (in alpha dim) 
super_resolution_factor = instru.get_srf(
    [ch1c.det_pix_size],
    step_angle*3600, # Conversion in arcsec
)

alpha_axis = origin_alpha_axis - np.mean(origin_alpha_axis) + targ_ra
beta_axis = origin_beta_axis - np.mean(origin_beta_axis) + targ_dec
pointings = instru.CoordList([orgin_target]).pix(step_angle)
from surfh.DottestModels import MCMO_SigRLSCT_Channel_Model
# Channel 
channels = [
            MCMO_SigRLSCT_Channel_Model.Channel(
                ch1c,
                alpha_axis,
                beta_axis,
                wavelength_1c,
                super_resolution_factor[0],
                pointings,
                step_angle
            )
        ]

2024-08-09 16:33:10.930 | INFO     | surfh.Models.instru:pix:684 - IFU. 1C pixelized to 6.9e-06


step =  0.025
[0.196]


2024-08-09 16:33:12.051583: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [11]:
print(f'Slit Shape channel 1C : {channels[0].oshape}')
print(ch1c.slit_fov[1].local.alpha_width)
print(f'Shape of Channel 1C output is ')

Slit Shape channel 1C : (1, 21, 1400, 19)
0.0008888888888888889
Shape of Channel 1C output is 
